# Reading and merging data

In [3]:
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings("ignore")

In [10]:
# Data from 09/2020 to 12/2020 for high volume fhv

fhvhv_0920_feather = pd.read_feather('/Volumes/E/data/feather/fhvhv_tripdata_2020-09.feather')
fhvhv_1020_feather = pd.read_feather('/Volumes/E/data/feather/fhvhv_tripdata_2020-10.feather')
fhvhv_1120_feather = pd.read_feather('/Volumes/E/data/feather/fhvhv_tripdata_2020-11.feather')
fhvhv_1220_feather = pd.read_feather('/Volumes/E/data/feather/fhvhv_tripdata_2020-12.feather')


In [11]:
# Merge the data from Sep to Dec

frames = [fhvhv_0920_feather, fhvhv_1020_feather, fhvhv_1120_feather, fhvhv_1220_feather]
fhvhv_2020_df = pd.concat(frames, keys=["Sep", "Oct", "Nov", "Dec"])
fhvhv_2020_df.head()

hvfhs_license_num dispatching_base_num      pickup_datetime  \
Sep 0            HV0003               B02764  2020-09-01 00:14:27   
    1            HV0003               B02764  2020-09-01 00:31:36   
    2            HV0003               B02682  2020-09-01 00:19:08   
    3            HV0003               B02764  2020-09-01 00:32:09   
    4            HV0003               B02878  2020-09-01 00:06:58   

          dropoff_datetime  PULocationID  DOLocationID  SR_Flag  
Sep 0  2020-09-01 00:20:02           170           230      NaN  
    1  2020-09-01 00:52:06           163           265      NaN  
    2  2020-09-01 00:23:52           223           223      NaN  
    3  2020-09-01 01:08:58             4           265      NaN  
    4  2020-09-01 00:24:47           244           254      NaN

In [23]:
# Data that records covid cases by day in nyc

covid_df = pd.read_csv('/Volumes/E/data/cases-by-day.csv')
covid_df.head()

,date_of_interest,CASE_COUNT,PROBABLE_CASE_COUNT,CASE_COUNT_7DAY_AVG,ALL_CASE_COUNT_7DAY_AVG,BX_CASE_COUNT,BX_PROBABLE_CASE_COUNT,BX_CASE_COUNT_7DAY_AVG,BX_ALL_CASE_COUNT_7DAY_AVG,BK_CASE_COUNT,...,MN_ALL_CASE_COUNT_7DAY_AVG,QN_CASE_COUNT,QN_PROBABLE_CASE_COUNT,QN_CASE_COUNT_7DAY_AVG,QN_ALL_CASE_COUNT_7DAY_AVG,SI_CASE_COUNT,SI_PROBABLE_CASE_COUNT,SI_CASE_COUNT_7DAY_AVG,SI_ALL_CASE_COUNT_7DAY_AVG,INCOMPLETE
0,02/29/2020,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,03/01/2020,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,03/02/2020,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,03/03/2020,1,0,0,0,0,0,0,0,0,...,0,1,0,0,0,0,0,0,0,0
4,03/04/2020,5,0,0,0,0,0,0,0,1,...,0,2,0,0,0,0,0,0,0,0


# Data cleaning and preprocessing

High Volume FHV data

In [5]:
fhvhv_2020_df.dtypes

hvfhs_license_num        object
dispatching_base_num     object
pickup_datetime          object
dropoff_datetime         object
PULocationID              int64
DOLocationID              int64
SR_Flag                 float64
dtype: object

In [12]:
fhvhv_2020_df.drop(columns=['dispatching_base_num'], inplace=True)

In [13]:
# Impute missing values for share rides 
fhvhv_2020_df['SR_Flag'].fillna(0, inplace=True)
fhvhv_2020_df.dropna(0, inplace = True)
print("Done filling missing values")

# Convert share rides to categorical data
fhvhv_2020_df['SR_Flag'] = fhvhv_2020_df['SR_Flag'].astype('bool')

# Check if any rows have missing values, if not, proceed
fhvhv_2020_df.isnull().sum().sort_values(ascending=False)

Done filling missing values


hvfhs_license_num    0
pickup_datetime      0
dropoff_datetime     0
PULocationID         0
DOLocationID         0
SR_Flag              0
dtype: int64

In [14]:
# Add a new column which comprises the duration of each trip in minutes
# Codes are modified based on:
# https://stackoverflow.com/questions/51938140/how-to-get-the-time-duration-from-two-date-time-columns-of-pandas-dataframe
# Might take up to 10 minutes to load...

covert_to_min = 60

fhvhv_2020_df['pickup_datetime'] = pd.to_datetime(fhvhv_2020_df['pickup_datetime'])
fhvhv_2020_df['dropoff_datetime'] = pd.to_datetime(fhvhv_2020_df['dropoff_datetime'])
fhvhv_2020_df['trip_duration_min'] = (fhvhv_2020_df['dropoff_datetime'] - fhvhv_2020_df['pickup_datetime']).apply(
    lambda x: x.seconds/(covert_to_min))
fhvhv_2020_df['trip_duration_min'] = fhvhv_2020_df['trip_duration_min'].round(2)

fhvhv_2020_df.head()

hvfhs_license_num     pickup_datetime    dropoff_datetime  PULocationID  \
Sep 0            HV0003 2020-09-01 00:14:27 2020-09-01 00:20:02           170   
    1            HV0003 2020-09-01 00:31:36 2020-09-01 00:52:06           163   
    2            HV0003 2020-09-01 00:19:08 2020-09-01 00:23:52           223   
    3            HV0003 2020-09-01 00:32:09 2020-09-01 01:08:58             4   
    4            HV0003 2020-09-01 00:06:58 2020-09-01 00:24:47           244   

       DOLocationID  SR_Flag  trip_duration_min  
Sep 0           230    False               5.58  
    1           265    False              20.50  
    2           223    False               4.73  
    3           265    False              36.82  
    4           254    False              17.82

In [15]:
# Remove outliers: minimum and maximum values of trip duration

outlier_min = fhvhv_2020_df['trip_duration_min'].min()
print('Minimum trip duration: ',outlier_min)
outlier_max = fhvhv_2020_df['trip_duration_min'].max()
print('Maximum trip duration: ',outlier_max)

fhvhv_2020_df = fhvhv_2020_df[(fhvhv_2020_df.trip_duration_min != outlier_min)
                              & (fhvhv_2020_df.trip_duration_min != outlier_max)]    
print('Outliers removed!')

Minimum trip duration:  0.0
Maximum trip duration:  1439.88
Outliers removed!


In [16]:
# Analysis on weekday's and hourly pickup

fhvhv_2020_df['pickup_day'] = pd.DatetimeIndex(fhvhv_2020_df['pickup_datetime']).weekday
fhvhv_2020_df['pickup_hour'] = pd.DatetimeIndex(fhvhv_2020_df['pickup_datetime']).hour

fhvhv_2020_df.head()

hvfhs_license_num     pickup_datetime    dropoff_datetime  PULocationID  \
Sep 0            HV0003 2020-09-01 00:14:27 2020-09-01 00:20:02           170   
    1            HV0003 2020-09-01 00:31:36 2020-09-01 00:52:06           163   
    2            HV0003 2020-09-01 00:19:08 2020-09-01 00:23:52           223   
    3            HV0003 2020-09-01 00:32:09 2020-09-01 01:08:58             4   
    4            HV0003 2020-09-01 00:06:58 2020-09-01 00:24:47           244   

       DOLocationID  SR_Flag  trip_duration_min  pickup_day  pickup_hour  
Sep 0           230    False               5.58           1            0  
    1           265    False              20.50           1            0  
    2           223    False               4.73           1            0  
    3           265    False              36.82           1            0  
    4           254    False              17.82           1            0

COVID-19 data

In [17]:
covid_df.dtypes

date_of_interest              object
CASE_COUNT                     int64
PROBABLE_CASE_COUNT            int64
CASE_COUNT_7DAY_AVG            int64
ALL_CASE_COUNT_7DAY_AVG        int64
BX_CASE_COUNT                  int64
BX_PROBABLE_CASE_COUNT         int64
BX_CASE_COUNT_7DAY_AVG         int64
BX_ALL_CASE_COUNT_7DAY_AVG     int64
BK_CASE_COUNT                  int64
BK_PROBABLE_CASE_COUNT         int64
BK_CASE_COUNT_7DAY_AVG         int64
BK_ALL_CASE_COUNT_7DAY_AVG     int64
MN_CASE_COUNT                  int64
MN_PROBABLE_CASE_COUNT         int64
MN_CASE_COUNT_7DAY_AVG         int64
MN_ALL_CASE_COUNT_7DAY_AVG     int64
QN_CASE_COUNT                  int64
QN_PROBABLE_CASE_COUNT         int64
QN_CASE_COUNT_7DAY_AVG         int64
QN_ALL_CASE_COUNT_7DAY_AVG     int64
SI_CASE_COUNT                  int64
SI_PROBABLE_CASE_COUNT         int64
SI_CASE_COUNT_7DAY_AVG         int64
SI_ALL_CASE_COUNT_7DAY_AVG     int64
dtype: object

In [25]:
covid_df.drop(['INCOMPLETE'], axis=1, inplace=True)

In [32]:
# Only keep the data within the time period of interest
start_date = '2020-09-01'
end_date = '2020-12-31'

covid_df['date_of_interest'] = pd.to_datetime(covid_df['date_of_interest'])
time_period = (covid_df['date_of_interest'] >= start_date) & (covid_df['date_of_interest'] <= end_date)
covid_df = covid_df.loc[time_period].reset_index(drop=True)

# Descriptive Statistics

High Volume FHV data

In [14]:
fhvhv_2020_df.shape

(48556503, 10)

In [15]:
pd.options.display.float_format = '{:,.2f}'.format
fhvhv_2020_df.describe()

,PULocationID,DOLocationID,trip_duration_min,pickup_weekday,pickup_hour
count,"48,556,503.00","48,556,503.00","48,556,503.00","48,556,503.00","48,556,503.00"
mean,135.50,139.01,17.41,3.13,13.61
std,76.77,78.65,20.64,1.95,6.18
min,1.00,1.00,0.02,0.00,0.00
25%,70.00,72.00,9.32,1.00,9.00
50%,137.00,140.00,14.35,3.00,14.00
75%,210.00,215.00,21.87,5.00,19.00
max,265.00,265.00,"1,439.82",6.00,23.00


In [16]:
# Count the number of trips according to HVFHS business
# HV0002: Juno
# HV0003: Uber
# HV0004: Via
# HV0005: Lyft

print("Number of trips according to HVFHS business from 09/2020 to 12/2020:")
fhvhv_2020_df['hvfhs_license_num'].value_counts()

Number of trips according to HVFHS business from 09/2020 to 12/2020:


HV0003    35456920
HV0005    12551974
HV0004      547609
Name: hvfhs_license_num, dtype: int64

COVID-19 data

In [30]:
covid_df.describe()

,CASE_COUNT,PROBABLE_CASE_COUNT,CASE_COUNT_7DAY_AVG,ALL_CASE_COUNT_7DAY_AVG,BX_CASE_COUNT,BX_PROBABLE_CASE_COUNT,BX_CASE_COUNT_7DAY_AVG,BX_ALL_CASE_COUNT_7DAY_AVG,BK_CASE_COUNT,BK_PROBABLE_CASE_COUNT,...,MN_CASE_COUNT_7DAY_AVG,MN_ALL_CASE_COUNT_7DAY_AVG,QN_CASE_COUNT,QN_PROBABLE_CASE_COUNT,QN_CASE_COUNT_7DAY_AVG,QN_ALL_CASE_COUNT_7DAY_AVG,SI_CASE_COUNT,SI_PROBABLE_CASE_COUNT,SI_CASE_COUNT_7DAY_AVG,SI_ALL_CASE_COUNT_7DAY_AVG
count,122.000000,122.000000,122.000000,122.000000,122.000000,122.000000,122.000000,122.000000,122.000000,122.000000,...,122.000000,122.000000,122.000000,122.000000,122.000000,122.000000,122.000000,122.000000,122.000000,122.000000
mean,1384.385246,380.811475,1283.336066,1640.877049,233.262295,66.032787,214.008197,276.024590,417.622951,105.803279,...,193.163934,251.852459,385.057377,111.180328,354.360656,457.926230,143.032787,35.983607,132.163934,166.155738
std,1250.626081,343.916296,1067.219920,1386.571058,227.901088,62.467702,185.770208,243.091333,352.259280,92.243447,...,150.946455,201.936152,371.580727,107.683498,316.837004,416.019512,137.441554,32.000254,119.195676,148.069674
min,156.000000,11.000000,236.000000,259.000000,18.000000,1.000000,37.000000,42.000000,37.000000,3.000000,...,37.000000,41.000000,28.000000,3.000000,58.000000,62.000000,13.000000,0.000000,18.000000,19.000000
25%,417.500000,65.250000,470.250000,539.000000,63.250000,9.250000,59.250000,67.000000,168.000000,28.250000,...,64.500000,71.500000,104.500000,16.000000,116.750000,135.250000,31.250000,4.000000,31.000000,36.000000
50%,774.000000,234.000000,623.000000,826.000000,123.500000,39.500000,105.000000,137.500000,245.000000,71.500000,...,101.500000,142.500000,191.000000,61.500000,155.500000,204.500000,83.000000,31.000000,65.500000,92.000000
75%,2150.250000,639.250000,2094.000000,2760.250000,355.750000,112.750000,357.500000,476.500000,593.250000,177.750000,...,325.000000,445.250000,603.000000,188.500000,564.000000,749.750000,239.500000,63.000000,260.500000,323.500000
max,5385.000000,1196.000000,3761.000000,4682.000000,1019.000000,212.000000,687.000000,847.000000,1600.000000,346.000000,...,467.000000,588.000000,1602.000000,390.000000,1125.000000,1424.000000,584.000000,107.000000,400.000000,481.000000


# Data aggregation

In [31]:
# Prepare to aggregate fhvhv taxi data and covid data
# Takes about 5 min to load

# Keep attributes we are interested in
fhvhv_by_day_df = fhvhv_2020_df[['pickup_datetime','SR_Flag','trip_duration_min']]

# To match the datetime template in covid data
fhvhv_by_day_df['pickup_date'] = fhvhv_2020_df['pickup_datetime'].dt.strftime('%Y-%m-%d')

fhvhv_by_day_df.head()

pickup_datetime  SR_Flag  trip_duration_min pickup_date
Sep 0 2020-09-01 00:14:27    False               5.58  2020-09-01
    1 2020-09-01 00:31:36    False              20.50  2020-09-01
    2 2020-09-01 00:19:08    False               4.73  2020-09-01
    3 2020-09-01 00:32:09    False              36.82  2020-09-01
    4 2020-09-01 00:06:58    False              17.82  2020-09-01

In [33]:
# Aggregating all attributes by date
# Codes are modified based on:
# https://stackoverflow.com/questions/45752601/how-to-do-a-conditional-count-after-groupby-on-a-pandas-dataframe


reduced_df = fhvhv_by_day_df.groupby('pickup_date')['SR_Flag'].apply(
    lambda x: (x==True).sum()).reset_index(name='num_of_SR')
reduced_df['trip_count'] = fhvhv_by_day_df.groupby('pickup_date')['pickup_date'].agg('count').reset_index(
    name='trip').trip
reduced_df['avg_trip_duration'] = fhvhv_by_day_df.groupby('pickup_date')['trip_duration_min'].agg('mean').reset_index(
    name='duration').duration
reduced_df.head()

,pickup_date,num_of_SR,trip_count,avg_trip_duration
0,2020-09-01,34,344879,17.833136
1,2020-09-02,42,357969,17.718040
2,2020-09-03,26,391021,18.258425
3,2020-09-04,6,425570,19.115868
4,2020-09-05,0,434656,17.152276


In [34]:
# Combine the fhvhv data with covid data

fhvhv_covid_df = pd.merge(reduced_df,covid_df,left_index=True,right_index=True)
fhvhv_covid_df = fhvhv_covid_df.drop(columns=['date_of_interest'])
fhvhv_covid_df

,pickup_date,num_of_SR,trip_count,avg_trip_duration,CASE_COUNT,PROBABLE_CASE_COUNT,CASE_COUNT_7DAY_AVG,ALL_CASE_COUNT_7DAY_AVG,BX_CASE_COUNT,BX_PROBABLE_CASE_COUNT,...,MN_CASE_COUNT_7DAY_AVG,MN_ALL_CASE_COUNT_7DAY_AVG,QN_CASE_COUNT,QN_PROBABLE_CASE_COUNT,QN_CASE_COUNT_7DAY_AVG,QN_ALL_CASE_COUNT_7DAY_AVG,SI_CASE_COUNT,SI_PROBABLE_CASE_COUNT,SI_CASE_COUNT_7DAY_AVG,SI_ALL_CASE_COUNT_7DAY_AVG
0,2020-09-01,34,344879,17.833136,294,34,255,278,52,9,...,45,48,73,4,58,62,24,1,18,19
1,2020-09-02,42,357969,17.718040,271,25,249,272,42,6,...,45,48,69,3,58,62,17,0,18,19
2,2020-09-03,26,391021,18.258425,274,29,250,274,41,5,...,43,46,90,11,62,67,17,4,19,20
3,2020-09-04,6,425570,19.115868,230,19,243,267,44,1,...,40,43,59,4,63,68,26,2,20,21
4,2020-09-05,0,434656,17.152276,156,11,247,269,30,3,...,40,43,34,3,63,68,24,0,22,23
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
117,2020-12-27,0,285895,15.374107,2886,920,3239,4094,479,170,...,428,540,868,317,970,1238,312,79,319,395
118,2020-12-28,6,301203,16.477568,5385,1196,3389,4266,1019,212,...,438,553,1530,390,1005,1284,562,89,338,415
119,2020-12-29,2,330461,16.430484,5201,1003,3547,4432,991,196,...,448,562,1602,318,1062,1347,584,80,369,447
120,2020-12-30,2,358647,16.910249,4975,1052,3708,4610,966,182,...,467,588,1492,328,1105,1400,465,92,379,457


# Saving preprocessed data

In [35]:
fhvhv_2020_df = fhvhv_2020_df.reset_index(drop=True)
fhvhv_2020_df.to_feather(r'/Volumes/E/data/preprocessed_data/fhvhv_2020.feather')

fhvhv_covid_df.to_csv(r'/Volumes/E/data/preprocessed_data/fhvhv_covid.csv',index=False)